# Cooperative Interaction on JudgeBench with Position Swapping


## Imports

In [11]:
import pandas as pd
from datasets import load_dataset
import os
from dotenv import load_dotenv
from autogen import ConversableAgent, GroupChat, GroupChatManager
import re
from tqdm import tqdm

## Data

In [12]:
JudgeBench_Claude = load_dataset("ScalerLab/JudgeBench", split="claude")

df = pd.DataFrame(JudgeBench_Claude)

df_sampled = df.iloc[:270].sample(n=100, random_state=42).reset_index(drop=True)

df_final = df_sampled[["question", "response_A", "response_B", "label"]]

print(df_final.info())
print(df_final.head(1))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    100 non-null    object
 1   response_A  100 non-null    object
 2   response_B  100 non-null    object
 3   label       100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB
None
                                            question  \
0  Under standard temperature and pressure condit...   

                                          response_A  \
0  Let's approach this step-by-step:\n\n1) The ra...   

                                          response_B label  
0  Let's approach this step-by-step:\n\n1) The ra...   B>A  


## Config

In [13]:
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_API_VERSION", "2023-12-01-preview")

config_list = [
    {
        "model": deployment_name,
        "api_key": api_key,
        "base_url": f"{endpoint}/openai/deployments/{deployment_name}/chat/completions?api-version={api_version}",
        "api_type": "azure",
        "api_version": api_version,  
        "temperature": 0,
        "cache_seed": 42
    }
]

llm_config={"config_list": config_list}

## System Design

In [ ]:
agent_1_system_message =f"""
You are an Analyst-Agent in a cooperative reasoning chain.
Your task is to provide a factual analysis of both Response A and Response B:
1. Extract the main claims made in each response.
2. Extract the supporting arguments for each claim.
3. Highlight any logical gaps or missing justification.
4. Give an initial assessment whether Response A or Response B is more factually correct.
Output your analysis in a structured format using around 200 words.
Always begin your output with: "As an Analyst-Agent I think ..."
Always end your output with: "In an initial assessment I conclude ..."
Always end your output by stating whether Response A or B is more factually correct using the following JSON format: {{"response": A/B}}
"""

agent_2_system_message =f"""
You are a Critical-Enhancer-Agent in a cooperative reasoning chain.
Your task is to carefully review the Analyst-Agent's analysis and improve upon it.
You may add missing facts, point out inaccuracies or refine the reasoning in a constructive way.
Use around 200 words for your analysis.
Always begin your output with: "Building on the Analyst-Agent's initial assessment, I ..."
Always end your output by stating whether Response A or B is more factually correct using the following JSON format: {{"response": A/B}}
"""

agent_3_system_message =f"""
You are the Decision-Agent in a cooperative reasoning chain.
Your task is to integrate the insights from the Analyst-Agent and the Critical-Enhancer-Agent into one response.
Resolve any contradictions and correct factual mistakes.
Give an explanation for your final conclusion.
Always end your output by stating whether Response A or B is more factually correct using the following JSON format: {{"response": A/B}}
"""

In [15]:
agent_1 = ConversableAgent(
    name="Analyst-Agent",
    llm_config=llm_config,
    system_message=agent_1_system_message,
    human_input_mode="NEVER",
)

agent_2 = ConversableAgent(
    name="Critical-Enhancer-Agent",
    llm_config=llm_config,
    system_message=agent_2_system_message,
    human_input_mode="NEVER",
)

agent_3 = ConversableAgent(
    name="Decision-Agent",
    llm_config=llm_config,
    system_message=agent_3_system_message,
    human_input_mode="NEVER",
)

group_chat = GroupChat(
    agents=[agent_1, agent_2, agent_3],
    messages=[],
    max_round=4,
    speaker_selection_method="round_robin"
)

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
)

## Evaluation

In [16]:
def evaluate(question, response_A, response_B, label):

    message = f""" 
    Question: {question}

    Response A: {response_A}

    Response B: {response_B}
    """

    chat_results = agent_3.initiate_chat(
        group_chat_manager,
        message=message,
        summary_method="last_msg",
    )

    result_str = str(chat_results.chat_history[-1]["content"])
    print(result_str)

    pattern = r'"response"\s*:\s*"?([AB])"?'

    match = re.search(pattern, result_str)
    system_decision = match.group(1) if match else "X"
    print(system_decision)

    ground_truth = "A" if "A>" in label else "B"

    is_correct = system_decision == ground_truth

    return {
        "system_decision": system_decision,
        "ground_truth": ground_truth,
        "is_correct": is_correct
    }

In [17]:
num_rows = 10

df_final_subset = df_final.head(num_rows)

# normal position
results_1 = []

for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_A"],
        response_B=row["response_B"],
        label=row["label"],
    )
    results_1.append(result)

results_1_df = pd.DataFrame(results_1)
results_1_df.to_csv('Results/cooperative_1.csv', index=False)

accuracy_1 = results_1_df["is_correct"].mean()
print(f"Accuracy_1: {accuracy_1:.2%}")

position_distribution_1 = results_1_df["system_decision"].value_counts(normalize=True) * 100
print(f"Position_Distribution_1: {position_distribution_1}")

# swapped position
results_2 = []

for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_B"], # positions swap 
        response_B=row["response_A"], # positions swap 
        label=row["label"],
    )

    # position swap back
    if result["system_decision"] == "A":
        result["system_decision"] = "B"
    elif result["system_decision"] == "B":
        result["system_decision"] = "A"

    if result["is_correct"] == True:
        result["is_correct"] = False
    elif result["is_correct"] == False:
        result["is_correct"] = True

    results_2.append(result)

results_2_df = pd.DataFrame(results_2)
results_2_df.to_csv('Results/cooperative_2.csv', index=False)

accuracy_2 = results_2_df["is_correct"].mean()
print(f"Accuracy_2: {accuracy_2:.2%}")

position_distribution_2 = results_2_df["system_decision"].value_counts(normalize=True) * 100
print(f"Position_Distribution_2: {position_distribution_2}")


Progress:   0%|          | 0/10 [00:00<?, ?it/s]

Decision-Agent (to chat_manager):

 
    Question: Under standard temperature and pressure conditions, compare the relative rates at which inert gases,Ar, He, and Kr diffuse through a common orifice.
(A) .1002 : .3002 : .4002
(B) .3582 : .4582 : .0582
(C) .2582 : .4998 : .3092
(D) .1582 : .6008 : .2092
(E) .1582 : .4998 : .1092
(F) .2002 : .4002 : .1092
(G) .1582 : .3998 : .2592
(H) .2502 : .4502 : .1502
(I) .2082 : .5998 : .1592
(J) .1802 : .4802 : .2802
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's approach this step-by-step:

1) The rate of diffusion of gases is inversely proportional to the square root of their molecular masses. This is known as Graham's Law.

2) We need to find the relative rates, so we'll use the formula: 
   Rate ∝ 1/√(Molecular Mass)

3) Mole


Next speaker: Analyst-Agent

Analyst-Agent (to chat_manager):

As an Analyst-Agent I think the following analysis can be made regarding Response A and Response B:

**Response A:**
1. **Main Claims:** The response claims that the relative rates of diffusion for Ar, He, and Kr are best represented by option D (0.1582 : 0.6008 : 0.2092).
2. **Supporting Arguments:** It uses Graham's Law to calculate the diffusion rates based on molecular masses, providing specific calculations for each gas and normalizing the results.
3. **Logical Gaps:** The final normalized values do not match the option D provided, indicating a potential miscalculation or misinterpretation of the options.
   
**Response B:**
1. **Main Claims:** The response claims that the relative rates of diffusion for Ar, He, and Kr are best represented by option E (0.1582 : 0.4998 : 0.1092).
2. **Supporting Arguments:** Similar to Response A, it applies Graham's Law and provides calculations for the diffusion rates, normalizing th

Progress:  10%|█         | 1/10 [00:00<00:01,  6.66it/s]

After integrating the insights from both the Analyst-Agent and the Critical-Enhancer-Agent, we can clarify the analysis of the diffusion rates of the inert gases Ar, He, and Kr.

1. **Graham's Law Application:** Both responses correctly apply Graham's Law, which states that the rate of diffusion is inversely proportional to the square root of the molecular mass. This principle is fundamental in determining the relative rates of diffusion for the gases in question.

2. **Molecular Masses:** The molecular masses provided in both responses are accurate:
   - Ar (Argon): 39.95 g/mol
   - He (Helium): 4.00 g/mol
   - Kr (Krypton): 83.80 g/mol

3. **Calculating Diffusion Rates:** The calculations for the diffusion rates are as follows:
   - Ar: \( \frac{1}{\sqrt{39.95}} \approx 0.1582 \)
   - He: \( \frac{1}{\sqrt{4.00}} = 0.5000 \)
   - Kr: \( \frac{1}{\sqrt{83.80}} \approx 0.1092 \)

4. **Normalization Process:** The total of these rates is:
   - Total = \( 0.1582 + 0.5000 + 0.1092 = 0.767

...

After integrating the insights from both the Analyst-Agent and the Critical-Enhancer-Agent, it is clear that Response A is more factually correct. 

Response A accurately sets up the problem with the correct initial conditions and correctly applies the decay formulas for both isotopes. It correctly concludes that the ratio of atoms A to B will be reversed after 12 days, interpreting the target ratio correctly as A being half of B (1:2) from the initial 2:1 ratio.

In contrast, Response B misinterprets the target ratio, leading to an incorrect conclusion of 6 days. The misunderstanding of the ratio and the unnecessary complexity introduced by logarithmic calculations further detracts from its accuracy.

Thus, the final conclusion is that Response A is the correct answer.

{"response": "A"}
A
Decision-Agent (to chat_manager):

 
    Question: Compute the sample standard deviation of ${9, 14, 5, 4, -20, -13, -5, 13}$. Please put your final answer in a $\\boxed{}$.

    Response A: Let's a

Progress:  90%|█████████ | 9/10 [00:01<00:00,  6.65it/s]

After integrating the insights from both the Analyst-Agent and the Critical-Enhancer-Agent, we can summarize the findings as follows:

1. **Mean Calculation**: Both responses correctly calculate the mean as 0.875.

2. **Squared Differences**: 
   - Response A correctly computes the squared differences for all data points.
   - Response B contains an error in the squared difference for the data point 14, which should be 171.390625 instead of 171.890625.

3. **Sum of Squared Differences**: 
   - Response A incorrectly states the sum of squared differences as 1073.875. The correct sum, based on accurate squared differences, should be 1073.515625.

4. **Final Calculation**: 
   - Dividing the corrected sum of squared differences (1073.515625) by (n-1) = 7 gives approximately 153.359375. Taking the square root yields a sample standard deviation of approximately 12.37.

In conclusion, while Response A is closer to the correct answer, both responses contain errors. The final answer should be 

Progress: 100%|██████████| 10/10 [00:01<00:00,  7.25it/s]

After integrating the insights from both the Analyst-Agent and the Critical-Enhancer-Agent, we can conclude the following:

1. **Main Claims**: Both Response A and Response B correctly implement a solution to find the maximum sum of a balanced subsequence using a Binary Indexed Tree (Fenwick Tree). They both utilize a similar approach with tuples of (nums[i] - i, i) for sorting and querying.

2. **Complexity Analysis**: Both responses should explicitly state that the overall time complexity of their algorithms is O(n log n). This includes O(n log n) for sorting and O(log n) for each update and query operation on the Binary Indexed Tree.

3. **Balanced Condition Explanation**: The transformation to (nums[i] - i, i) is crucial as it ensures that the condition for a balanced subsequence is maintained. This transformation captures the necessary differences between the values and their indices, allowing the algorithm to select valid subsequences.

4. **Coordinate Compression Clarification**